<a href="https://colab.research.google.com/github/RickyDoan/DL-Tensor-Flow-Prediction/blob/main/TF_CNN_flowers_Images_Classification_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import PIL
from tensorflow import keras
import pathlib
import cv2
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
# flower_photo/
#   daisy/
#   dandelion/
#   roses/
#   sunflowers/
#   tulips/

In [ ]:
# # dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
# dataset_url =
# data_dir = tf.keras.utils.get_file('flower_photos', cache_dir='/content/drive/MyDrive/A Learning Tensor Flow/CNN-Image Classification', origin=dataset_url, untar=True)

In [ ]:
# data_dir

In [ ]:
data_dir = "/content/drive/MyDrive/A Learning Tensor Flow/CNN-Image Classification/datasets/flower_photos"

In [ ]:
data_dir = pathlib.Path(data_dir)
data_dir

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(10, 10))
axes = axes.flatten()
for i in range(5):
    image = PIL.Image.open(str(roses[i]))
    axes[i].imshow(image)
    axes[i].set_xticks([])
    axes[i].set_yticks([])
    axes[i].grid(False)
plt.show()

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
PIL.Image.open(str(tulips[0]))

In [ ]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [ ]:
flowers_images_dict['roses'][:5]

In [ ]:
str(flowers_images_dict['roses'][0])

In [ ]:
img = cv2.imread(str(flowers_images_dict['roses'][0]))
img

In [ ]:
img.shape

In [ ]:
# X = np.array(X)
# y = np.array(y)

In [ ]:
# X.shape

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

In [ ]:
class_name = train_ds.class_names
print(class_name)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Training mode

In [ ]:
num_classes = len(class_name)
model = keras.Sequential([
    keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(num_classes)
])

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

with tf.device('/device:GPU:0'):
    model.fit(train_ds, validation_data=val_ds, epochs=10)
model.evaluate(val_ds)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
data_augumentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),
])

In [ ]:
num_classes = len(class_name)
model_aug = keras.Sequential([
    data_augumentation,
    keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    keras.layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(num_classes)
])

model_aug.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

with tf.device('/device:GPU:0'):
    history = model_aug.fit(train_ds, validation_data=val_ds, epochs=30)
model_aug.evaluate(val_ds)

In [ ]:
epochs = 30
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
class_names = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [ ]:
# prompt: create code to predict 10 random image from my folder.

import random
import matplotlib.pyplot as plt
import cv2
import os

# Assuming 'data_dir' and 'model_aug' are defined from the previous code.
# If not, define them as shown in the previous code block.

def predict_random_images(num_images=20):
    """Predicts the class of random images from the dataset and displays them."""

    random_image_paths = random.sample(list(data_dir.glob('*/*.jpg')), num_images)
    fig, axes = plt.subplots(4,5, figsize=(20, 10))
    axes = axes.flatten()
    for idx, image_path in enumerate(random_image_paths):
        img = cv2.imread(str(image_path))
        img = cv2.resize(img, (img_height, img_width))  # Resize the image
        img_array = tf.keras.utils.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) # Create a batch

        predictions = model_aug.predict(img_array)
        predicted_class = class_names[np.argmax(predictions[0])]

        # Get the true label from the image path
        true_label = os.path.basename(os.path.dirname(str(image_path)))

        # plt.figure()
        axes[idx].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axes[idx].set_title(f"Pred: {predicted_class}, Label : {true_label} ")
        axes[idx].axis("off")
    plt.tight_layout
    plt.show()

predict_random_images()

In [ ]:
from sklearn.metrics import classification_report

predict = model_aug.predict(val_ds)
predict = np.argmax(predict, axis=1)
true = np.concatenate([y for x, y in val_ds], axis=0)
print(classification_report(true, predict, target_names=class_name))

In [ ]:
# X = []
# y = []
# for flowers_name, images in flowers_images_dict.items():
#     for image in images:
#         img = cv2.imread(str(image))
#         resized_img = cv2.resize(img, (180, 180))
#         X.append(resized_img)
#         y.append(flowers_labels_dict[flowers_name])

In [ ]:
# X = np.array(X)
# y = np.array(y)

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

In [ ]:
# X_train = X_train/255
# X_test = X_test/255

In [ ]:
# data_augument = Sequential([
#     layers.RandomFlip("horizontal", input_shape=(180, 180, 3)),
#     layers.RandomRotation(0.1),
#     layers.RandomZoom(0.1),
# ])

In [ ]:
# num_classes = len(class_name)
# model_augumentation = Sequential([
#     data_augument,
#     layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu'),
#     layers.MaxPooling2D(pool_size=(2, 2)),
#     layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),
#     layers.MaxPooling2D(pool_size=(2, 2)),
#     layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
#     layers.MaxPooling2D(pool_size=(2, 2)),
#     layers.Dropout(0.2),
#     layers.Flatten(),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(num_classes)
# ])
# model_augumentation.compile(optimizer='adam',
#               loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# with tf.device('/device:GPU:0'):
#     model_augumentation.fit(X_train, y_train, epochs=30)
# model_augumentation.evaluate(X_test, y_test)